### Входные данные

У вас имеется поток данных (генератор data_stream). Поля это случайные величины - так сделано для упрощения генерации данных. Есть три поля (названы по уровню сложности задания)

### Задание
##### Мотивация:
У вас есть куча временных рядов, вы хотите научиться предсказывать следующее значение по 1000 предыдущим. 1000 признаков окна это слишком много, однако вы решили заменить их 5ю: средним, дисперсией, минимумом, медианой и максимумом. Однако, все эти признаки надо подсчитать, причём хочется уметь это делать быстро (в течение часа)
##### Для каждого поля нужно сделать следующее:

1. Пробежаться по данным окном размера 1000 (окно сдвигается на 1, то есть следующее окно пересекается с предыдущим по 999 элементам).

2. Для каждого окна посчитайте среднее значение поля и его дисперсию. Делайте yield этих значений, получая генератор tuple. 

3. Для каждого окна найдине минимум, медиану и максимум в нём. Делайте yield этих значений, получая генератор tuple. 

Ответом, который нужно будет засабмитить в гугл форму, является среднее значение tuple по получившемуся потоку, округлённое до 2го знака.

### Замечания

1. Обратите внимания как генерируются поля. Постарайтесь понять особенность каждого поля и как это можно использовать. Желательно, чтобы для каждого поля у вас было своё решение, максимально эффективно использующее знание об этом поле.
2. Полезные библиотеки: itertools, numpy, collections + всё что найдёте в интернете и можно поставить через pip install
3. **Медианой отсортированного массива arr считайте значение arr[len(arr) // 2]**



Если измерять время работы функций временем работы функции example, то примерное время работы такое:
Одновременно среднее, дисперсия - 1.17
Одновременно минимум, максимум и медиана:easy - 0.87
medium - 2.11
nightmare - 2.85


#### Генерация данных

In [61]:
from collections import namedtuple, deque
import random
from decimal import Decimal

Record = namedtuple('Record', 'easy medium nightmare')

def data_stream():
    random_generator = random.Random(42)
    easy = 0
    for _ in range(10000000):
        easy += random_generator.randint(0, 2) 
        medium = random_generator.randint(0, 256 - 1)
        nightmare = random_generator.randint(0, 1000000000 - 1)
        
        yield Record(
            easy=easy,
            medium=medium,
            nightmare=nightmare
        )
        
def easy_stream():
    for record in data_stream():
        yield record.easy
        
def medium_stream():
    for record in data_stream():
        yield record.medium
        
def nightmare_stream():
    for record in data_stream():
        yield record.nightmare

#### Подсчёт среднего значения tuple по потоку

In [62]:
import numpy as np

def get_tuple_stream_mean(stream, number_of_values):
    result = np.zeros(number_of_values, dtype=np.dtype(Decimal))
    count = Decimal(0) 
    for streamed_tuple in stream:
        result += streamed_tuple
        count += 1
    #return ['{:0.2f}'.format(x) for x in result / count]
    return result / count

In [63]:
%%time
def example(stream):
    for value in stream:
        yield (value, value + 10)
print(get_tuple_stream_mean(example(medium_stream()), 2))

[Decimal('127.4812041') Decimal('137.4812041')]
CPU times: user 1min 43s, sys: 8 ms, total: 1min 43s
Wall time: 1min 43s


In [64]:
# Генератор среднего значения и дисперсии для всех потоков
def MV_custom(stream, window_size): # оптимизированный расчет
    window = deque(range(0, window_size), window_size) # создаем очередь для заполнения окна
    gen = stream
    count = 0 # счетчик для подсчета количества элементов в потоке и для отслеживания наполнения окна
    X_0 = 0 # первое число в очереди, которое будет вытесняся следующим числом из потока
    Mean_prev = 0 # первое значение среднего
    Variance_prev = 0 # первое значение дисперсии
# Будем уточнять сумму элементов в окне, после добавления нового элемента. 
# Для этого из предыдущей суммы (суммы элементов предыдущего окна) вычтем элемент очереди, который удалили
# и добавим элемент, который добавили в очередь. Таким образом нам не нужно будет пересчитывать среднее и 
# использовать np.mean() на выборке из 1000 элементов
    for number in gen:
        count +=1
        window.append(number)
        if count < window_size: continue # если окно не заполнено, продолжаем его заполнять
        
        if count == window_size: # если окно только что заполнилось, то первый раз:
            Variance = Decimal(np.var(window)) # считаем дисперсию первый раз
            Mean = Decimal(np.mean(window)) # считаем среднее первый раз
            X_0 = Decimal(window[0]) # запоминаем первое число в очереди, потом оно вытеснится новым число из потока
                 
            yield (Mean, Variance)
            continue
            

# пересчитываем новое значение среднего для нового (текущего) окна. Не суммируя снова все элементы
        Mean_prev = Mean # сохраняем пердыдущее значение среднего
        Variance_prev = Variance # сохраняем предыдущее значение дисперсии
        Mean = Mean_prev + Decimal(window[-1] - X_0)/ Decimal(window_size)
        Variance = Variance_prev + (Decimal(window[-1]) - X_0) * (Decimal(window[-1]) + X_0 - Mean - Mean_prev) / Decimal(window_size)
        
        # расчет в лоб, с помощью Numpy
        #Mean = np.mean(window)
        #Variance = np.var(window)
        
            
        #print("Window:", window)
        #print("X_0:", X_0, "\tX_n:", window[-1])
        #print("Mean_prev:", Mean_prev, "\tMean:", Mean)
        #print("Variance_prev:", Variance_prev, "\tVariance:", Variance, "Real Variance:", np.var(window), "\n")
        
        
        X_0 = window[0] # сохраняем значение самого левого элемента, перед тем как он будет вытеснен из очереди
        yield (Mean, Variance)    

In [65]:
# Генератор минимума, медианы и максимума для потока easy
def MMM_easy(window_size):
    window = deque(range(0, window_size), window_size) # создаем очередь для заполнения окна
    gen = easy_stream()
    count = 0 # счетчик для подсчета количества элементов в потоке и для отслеживания наполнения окна
    for d in gen:
        count +=1
        window.append(d)
        if count >= window_size: # если окно заполнено, то начинаем считать статистики
            easy_min = window[0]
            easy_median = window[(len(window))//2]
            easy_max = window[-1]
            yield (easy_min, easy_median, easy_max)

In [66]:
# Расчет Min, Median, Max (МММ) для потоков medium и nightmare. Эти потоки не сортированы по возрастанию, поэтому
# нужно включить сортировку и оптимизировать поиск медианы.

def MMM_brute(stream, window_size):
    window = deque(range(0, window_size), window_size) # создаем очередь для заполнения окна
    gen = stream
    count = 0 # счетчик для подсчета количества элементов в потоке и для отслеживания наполнения окна
    Min = 0
    Mean = 0
    Max = 0
    for d in stream:
        window.append(d)
        count += 1
        if count < window_size: 
            continue
        if count == window_size:
            sort_list = np.sort(window)
            Min = sort_list[0]
            Mean = sort_list[len(sort_list) // 2]
            Max = sort_list[-1]
            yield (Min, Mean, Max)
       
        sort_list = np.sort(window)
        Min = sort_list[0]
        Mean = sort_list[len(sort_list) // 2]
        Max = sort_list[-1]
# Отладочный блок, можно удалить
        #print("Window:", window)
        #print("Sorted window:", sort_list)
        #print("Min:", Min, "\tMean:", Mean, "\tMax:", Max)
        
        yield (Min, Mean, Max)

def MMM_custom(stream, window_size):
# Суть метода - создаем "зеркало" "окна", в котором выполняем сортировку и хранием его все время отсортированным.
# При поступлении из потока в "окно" нового значения, мы вставляем его в "зеркало" в соотвествии с ранжировкой,
# в свою очередь вбывшее из "окна" значение удаляем из "зеркала", сохраняя упорядоченность элементов в "зеркале".
# Таким образом, "окно" нам нужно только для того, что бы знать значение выбывшего и прибывшего элементов, 
# а минимум, максиммум, и медиану мы находим в "зеркале".
    
    window = deque(range(0, window_size), window_size) # создаем очередь для заполнения окна
    gen = stream
    count = 0 # счетчик для подсчета количества элементов в потоке и для отслеживания наполнения окна
    Min = 0
    Mean = 0
    Max = 0
    X_lost = 0 # значение, которое будет вытесняться из очереди следующим числом из потока
    for d in stream:
        window.append(d)
        count += 1
        if count < window_size: 
            continue
        if count == window_size: # если окно заполнилось, делаем его сортировку и сохраняем в зеркале
            mirrow = deque(np.sort(window), len(window)) # создаем массив для хранения "зеркала".
            Min = mirrow[0]
            Mean = mirrow[len(window) // 2]
            Max = mirrow[-1]
            X_lost = window[0] # сохраняем значение, которое будет вытесняться следующим числом из потока
            yield (Min, Mean, Max)
            continue
        
        mirrow.remove(X_lost)
        if window[-1] >= mirrow[-1]: 
            mirrow.append(window[-1])
        elif window[-1] <= mirrow[0]:
            mirrow.appendleft(window[-1])
        else:
            for i, _ in enumerate(mirrow):
                if mirrow[i] <= window[-1] <= mirrow[i+1]: 
                    mirrow.insert(i+1, window[-1])
                    break 
        X_lost = window[0]
        Min = mirrow[0]
        Mean = mirrow[len(mirrow) // 2]
        Max = mirrow[-1]
        yield (Min, Mean, Max)

In [67]:
%%time
stat_MV_easy = get_tuple_stream_mean(MV_custom(easy_stream(), 1000), 2)
#print(easy_MV_custom(test_gen(100), 10))

CPU times: user 2min 29s, sys: 38 ms, total: 2min 29s
Wall time: 2min 29s


In [81]:
print('Средние значения дисперсии и среднего арфиметического на потоке easy:\n')
print('Среднее значение mean -\t\t', stat_MV_easy[0])
print('Среднее значение variance -\t', stat_MV_easy[1])
print('Округленные значнения:', ['{:0.2f}'.format(x) for x in stat_MV_easy])
print(type(stat[0]))

Средние значения дисперсии и среднего арфиметического на потоке easy:

Среднее значение mean -		 4999675.276494780928603468009
Среднее значение variance -	 83439.33665582540527690643697
Округленные значнения: ['4999675.28', '83439.34']
<class 'decimal.Decimal'>


In [82]:
%%time
stat_MMM_easy = get_tuple_stream_mean(MMM_easy(1000), 3)

CPU times: user 2min, sys: 67 ms, total: 2min
Wall time: 2min


In [83]:
print('Средние значения Min, Median, Max на потоке easy:\n')
print('Среднее значение Min -\t\t', stat_MMM_easy[0])
print('Среднее значение Median -\t', stat_MMM_easy[1])
print('Среднее значение Max -\t', stat_MMM_easy[2])
print('Округленные значнения:', ['{:0.2f}'.format(x) for x in stat_MMM_easy])
print(type(stat[0]))

Средние значения Min, Median, Max на потоке easy:

Среднее значение Min -		 4999175.792842704986228124190
Среднее значение Median -	 4999675.776641486484499801530
Среднее значение Max -	 5000174.759596183658747508876
Округленные значнения: ['4999175.79', '4999675.78', '5000174.76']
<class 'decimal.Decimal'>


In [84]:
%%time
stat_MV_medium = get_tuple_stream_mean(MV_custom(medium_stream(), 1000), 2)

CPU times: user 2min 27s, sys: 5.94 ms, total: 2min 27s
Wall time: 2min 27s


In [85]:
print('Средние значения дисперсии и среднего арфиметического на потоке medium:\n')
print('Среднее значение mean -\t\t', stat_MV_medium[0])
print('Среднее значение variance -\t', stat_MV_medium[1])
print('Округленные значения:', ['{:0.2f}'.format(x) for x in stat_MV_medium])
print(type(stat[0]))

Средние значения дисперсии и среднего арфиметического на потоке medium:

Среднее значение mean -		 127.4811399705830630668178652
Среднее значение variance -	 5455.173897874811454295545482
Округленные значения: ['127.48', '5455.17']
<class 'decimal.Decimal'>


In [86]:
%%time
stat_MMM_medium = get_tuple_stream_mean(MMM_brute(medium_stream(),1000), 3)

CPU times: user 23min 18s, sys: 70 ms, total: 23min 18s
Wall time: 23min 18s


In [87]:
print('Средние значения Min, Median, Max на потоке medium:\n')
print('Среднее значение Min -\t\t', stat_MMM_medium[0])
print('Среднее значение Median -\t', stat_MMM_medium[1])
print('Среднее значение Max -\t', stat_MMM_medium[2])
print('Округленные значения:', ['{:0.2f}'.format(x) for x in stat_MMM_medium])
print(type(stat[0]))

Средние значения Min, Median, Max на потоке medium:

Среднее значение Min -		 0.01928982512454742983349738304
Среднее значение Median -	 127.6017212517809277365881115
Среднее значение Max -	 254.9790132055179106874866112
Округленные значения: ['0.02', '127.60', '254.98']
<class 'decimal.Decimal'>


In [88]:
%%time
stat_MV_nightmare = get_tuple_stream_mean(MV_custom(nightmare_stream(), 1000), 2)

CPU times: user 2min 42s, sys: 15 ms, total: 2min 42s
Wall time: 2min 42s


In [89]:
print('Средние значения дисперсии и среднего арфиметического на потоке nightmare:\n')
print('Среднее значение mean -\t\t', stat_MV_nightmare[0])
print('Среднее значение variance -\t', stat_MV_nightmare[1])
print('Округленные значения:', ['{:0.2f}'.format(x) for x in stat_MV_nightmare])
print(type(stat[0]))

Средние значения дисперсии и среднего арфиметического на потоке nightmare:

Среднее значение mean -		 499880345.8782329037258453391
Среднее значение variance -	 83228908564031124.12107455168
Округленные значения: ['499880345.88', '83228908564031124.12']
<class 'decimal.Decimal'>


Поле **nightmare**. _Дисперсия_

Правильные ответы

83228908564031114.59

83228908564031114.00

In [100]:
%%time
stat_MMM_nightmare = get_tuple_stream_mean(MMM_custom(nightmare_stream(),1000), 3)

CPU times: user 29min 12s, sys: 508 ms, total: 29min 13s
Wall time: 29min 14s


In [101]:
print('Средние значения Min, Median, Max на потоке nightmare:\n')
print('Среднее значение Min -\t\t', stat_MMM_nightmare[0])
print('Среднее значение Median -\t', stat_MMM_nightmare[1])
print('Среднее значение Max -\t', stat_MMM_nightmare[2])
print('Округленные значения:', ['{:0.2f}'.format(x) for x in stat_MMM_nightmare])
print(type(stat[0]))

Средние значения Min, Median, Max на потоке nightmare:

Среднее значение Min -		 1017525.922824723546200595993
Среднее значение Median -	 500438511.2309982199571038022
Среднее значение Max -	 999017352.9364254217342806720
Округленные значения: ['1017525.92', '500438511.23', '999017352.94']
<class 'decimal.Decimal'>


Поле nightmare. Минимум

Правильный ответ

1017512.29

Поле nightmare. Медиана

Правильный ответ

500438415.64

Поле nightmare. Максимум

Правильный ответ

999017359.97